In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import time
import utils
import Indicators as indicator
import Instruments as instrument


In [3]:
pair = 'GBP_JPY'
gran = 'H1'

In [4]:
df = pd.read_pickle(utils.get_candles_data_filename(pair, gran))

In [15]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,spread,ema_20,ema_50,atr_14
0,2020-08-23T23:00:00.000000000Z,1048,138.695,138.699,138.614,138.618,138.677,138.683,138.598,138.603,138.713,138.716,138.629,138.632,0.029,"ExponentialMovingWindow [com=0.5,min_periods=1...","ExponentialMovingWindow [com=0.5,min_periods=1...",NaN
1,2020-08-24T00:00:00.000000000Z,1958,138.620,138.686,138.420,138.433,138.604,138.670,138.403,138.416,138.636,138.704,138.435,138.450,0.034,"ExponentialMovingWindow [com=0.5,min_periods=1...","ExponentialMovingWindow [com=0.5,min_periods=1...",NaN
2,2020-08-24T01:00:00.000000000Z,1971,138.431,138.507,138.324,138.482,138.415,138.492,138.309,138.470,138.447,138.522,138.338,138.495,0.025,"ExponentialMovingWindow [com=0.5,min_periods=1...","ExponentialMovingWindow [com=0.5,min_periods=1...",NaN
3,2020-08-24T02:00:00.000000000Z,1012,138.484,138.516,138.436,138.475,138.472,138.502,138.421,138.458,138.497,138.531,138.449,138.492,0.034,"ExponentialMovingWindow [com=0.5,min_periods=1...","ExponentialMovingWindow [com=0.5,min_periods=1...",NaN
4,2020-08-24T03:00:00.000000000Z,878,138.478,138.528,138.464,138.468,138.462,138.513,138.451,138.455,138.493,138.543,138.477,138.481,0.026,"ExponentialMovingWindow [com=0.5,min_periods=1...","ExponentialMovingWindow [com=0.5,min_periods=1...",NaN


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    3999 non-null   object 
 1   volume  3999 non-null   int64  
 2   mid_o   3999 non-null   float64
 3   mid_h   3999 non-null   float64
 4   mid_l   3999 non-null   float64
 5   mid_c   3999 non-null   float64
 6   bid_o   3999 non-null   float64
 7   bid_h   3999 non-null   float64
 8   bid_l   3999 non-null   float64
 9   bid_c   3999 non-null   float64
 10  ask_o   3999 non-null   float64
 11  ask_h   3999 non-null   float64
 12  ask_l   3999 non-null   float64
 13  ask_c   3999 non-null   float64
 14  spread  3999 non-null   float64
dtypes: float64(13), int64(1), object(1)
memory usage: 468.8+ KB


In [5]:
remove = ['time', 'volume']
new_cols = [x for x in df.columns if x not in remove]
df[new_cols] = df[new_cols].apply(pd.to_numeric)

In [6]:
df['spread'] = df['ask_c'] - df['bid_c']
df.loc[0]

time      2020-08-23T23:00:00.000000000Z
volume                              1048
mid_o                            138.695
mid_h                            138.699
mid_l                            138.614
mid_c                            138.618
bid_o                            138.677
bid_h                            138.683
bid_l                            138.598
bid_c                            138.603
ask_o                            138.713
ask_h                            138.716
ask_l                            138.629
ask_c                            138.632
spread                             0.029
Name: 0, dtype: object

In [7]:
def results(candles, entries):
    cndl = 0
    openn = False
    results = []
    times = []
    while cndl < len(candles):
        # if we are not in a trade and we have a entry sign. Open a trade
        if entries[cndl] and not openn:
            enter = candles['ask_c'][cndl]
            stop = max((candles['atr_9'][cndl] * 0.5) - candles['ema_50'][cndl], enter-30)
            target = ((enter-stop) * 1.5) + candles['ask_c'][cndl]
            openn = True
            times.append(candles['time'][cndl])
            
        # if we are in a trade, monitor.
        elif openn:
            if candles['ask_h'][cndl] >= target:
                results.append(True)
                openn = False
            elif candles['ask_l'][cndl] <= stop:
                results.append(False)
                openn = False
        cndl += 1
        
    return times, results

In [69]:
def trade_finder(candles):
    # This function will determine all possible entries for the strategy specified below.
    
    # initialization
    pot_ent = [False] * len(candles)
    
    # list of booleans indicating if an entry will be made at the direct conclusion of a candle.
    # All start off False and as we go through the conditions, those will be switched to True for potential entries.
    # null = lambda (value) : value>0 or value<=0
    cndl = 0
    e20over50 = False
    reset = None
    while cndl < len(candles):        
        if cndl > 50:
            if candles['ema_20'][cndl] < candles['ema_50'][cndl]:
                e20over50 = False
                reset = False
                
            if candles['ema_20'][cndl] >= candles['ema_50'][cndl] and not reset:
                pot_ent[cndl] = True
                reset = True
            
            
        cndl += 1
    print(pot_ent)
    return results(candles, pot_ent)

    
    
        

In [52]:
indicator.exponential_moving_average(df, 'close', 20)

AttributeError: 'Series' object has no attribute 'emw'

In [46]:
def ema(candles, source, length, price='mid'):
    if source not in ['open', 'high', 'low', 'close']:
        print('invalid source')
        return None
    if type(length) != int or length < 1:
        print('invalid length')
        return None

    source_dict = {
        'open': 'o',
        'high': 'h',
        'low': 'l',
        'close': 'c'
    }
    
    ewm = candles[f'{price}_{source_dict[source]}'].ewm(span=length).mean()
    
    for a in range(length):
        ewm[a] = None
        
    candles[f'ema_{length}'] = ewm


In [47]:
ema(df, 'close', 20)
ema(df, 'close', 50)
indicator.average_true_range(df)

In [70]:
times, results = trade_finder(df)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False,

TypeError: 'list' object is not callable

In [49]:
df['ema_20']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
3994    150.035937
3995    150.013657
3996    150.010833
3997    150.008468
3998    149.995947
Name: ema_20, Length: 3999, dtype: float64

In [50]:
df['ema_50']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
3994    150.103069
3995    150.091263
3996    150.087056
3997    150.083093
3998    150.075011
Name: ema_50, Length: 3999, dtype: float64

In [44]:
for i in range(10):
    df['ema_50'][i] = None
    

<ipython-input-44-d77ec0e00b8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ema_50'][i] = None


In [26]:
df['mid_c'].ewm(span=20).mean().head(100)

0     138.618000
1     138.520875
2     138.506600
3     138.497478
4     138.490347
         ...    
95    140.417406
96    140.452513
97    140.482180
98    140.541499
99    140.615074
Name: mid_c, Length: 100, dtype: float64

In [38]:
a = df['mid_c'].ewm(span=50).mean().head(100)

0     138.618000
1     138.523650
2     138.509208
3     138.500136
4     138.493185
         ...    
95    140.104013
96    140.131322
97    140.156634
98    140.194548
99    140.239266
Name: mid_c, Length: 100, dtype: float64

In [55]:
res = []
for row in range(len(df)):
    if df['ema_50'][row] and df['ema_20'][row]:
        if df['ema_20'][row] >= df['ema_50'][row]:
            res.append(True)
        else:
            res.append(False)
             
res

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True

In [77]:
df['ema_50'][0]>0 or df['ema_50'][0]<=0

False